In [1]:
import pandas as pd
import re

claimsData = pd.read_csv('combined_csv.csv')
pd.set_option("display.max.columns", None) 

#Concatenates all of the sheets into a single sheet
under65 = pd.concat(pd.read_excel('State Under 65 Table 2018.xlsx',sheet_name=None), ignore_index=True)
over65 = pd.concat(pd.read_excel('State Over 65 Table 2018.xlsx',sheet_name=None), ignore_index=True)

cols = ['HCPCS Code', 'Minimum Submitted Charge Amount','Maximum Submitted Charge Amount','Standard Deviation of Submitted Charge Amount', 'Average Medicare Allowed Amount', 'Minimum Medicare Allowed Amount','Standard Deviation of Medicare Allowed Amount', 'Average Medicare Payment Amount', 'Minimum Medicare Payment Amount', 'Maximum Medicare Payment Amount','Standard Deviation of Medicare Payment Amount', 'Number of Providers', 'Number of Services', 'Number of Unique Beneficiary/Provider Interactions','Number of Distinct Medicare Beneficiary/Per Day Services', 'Average Submitted Charge Amount']

In [2]:
def onlyState(s):
    '''
    Function returns only the value that is within the parentheses
    '''
    # indexes the string from first "(" to the last ")" and returns value
    state = s[s.find("(")+1:s.find(")")]
    return state

In [3]:
def HCPCSgroup(HCPCS_Code):
    '''
    Assigns a HCPCS category depending on where in the code range that a value falls.
    Others category contains anything that is non-numeric (i.e. A1234)

    '''
    remove = re.compile(r"d\{5}")

    if HCPCS_Code >= 100 and HCPCS_Code <= 1999:
        return 'Anesthesia'
    elif HCPCS_Code >= 10004 and HCPCS_Code <= 69990:
        return 'Surgery'
    elif HCPCS_Code >= 70010 and HCPCS_Code <= 79999:
        return 'Radiology Procedures'
    elif HCPCS_Code >= 80047 and HCPCS_Code <= 89398:
        return 'Pathology and Laboratory Procedures'
    elif HCPCS_Code >= 90281 and HCPCS_Code <= 99756:
        return 'Medicine Services and Procedures'
    elif HCPCS_Code >= 99201 and HCPCS_Code <= 99499:
        return 'Evaluation and Management Services'
    elif HCPCS_Code != remove:
        return 'Others'
    else:
        return False 

In [4]:
def region(State):
    '''
    Assigns a region based on lists defined in beginning of notebook. 

    '''
    # defining different regions as a list to refer to...
    northEast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'New Jersey',             'Pennsylvania']

    midWest = ['Ohio', 'Michigan', 'Indiana', 'Wisconsin', 'Illinois', 'Minnesota', 'Iowa', 'Missouri', 'North Dakota', 'South Dakota',
    'Nebraska', 'Kansas']

    south = ['Delaware', 'Maryland', 'Virginia','West Virginia','Kentucky', 'North Carolina', 'South Carolina', 'Tennessee', 'Georgia',     'Florida', 'Alabama', 'Mississippi', 'Arkansas', 'Louisiana', 'Texas', 'Oklahoma', 'District of Columbia']

    west = ['Montana', 'Idaho', 'Wyoming', 'Colorado', 'New Mexico', 'Arizona', 'Utah', 'Nevada', 'California', 'Oregon', 'Washington',     'Alaska', 'Hawaii']

    other = ['American Samoa', 'Puerto Rico', 'Foreign Country', 'Armed Forces Europe', 'Virgin Islands', 'Armed Forces Central/South America','Guam', 'Unknown', 'Armed Forces Pacific', 'North Mariana Islands']

    if State in northEast:
        return 'North-East'
    elif State in midWest:
        return 'Mid-West'
    elif State in south:
        return 'South'
    elif State in west:
        return 'West'
    elif State in other:
        return 'Other'
    else:
        return 'No Location'

In [ ]:
# Drop the first national row
under65 = under65.drop(0)

# apply onlyState() to the 'State' dataframe
under65['State'] = under65.apply(lambda x: onlyState(x['State']), axis=1)

# apply region() to 'State' dataframe
Region = under65.apply(lambda x: region(x['State']), axis=1)

# create a new 'Region' dataframe and insert it into the 2nd column
under65.insert(1,'Region', Region, allow_duplicates=False)

In [ ]:
# Drop the first national row
over65 = over65.drop(0)

# apply onlyState() to the 'State' dataframe
over65['State'] = over65.apply(lambda x: onlyState(x['State']), axis=1)

# apply region() to 'State' dataframe
Region = over65.apply(lambda x: region(x['State']), axis=1)

# create a new 'Region' dataframe and insert it into the 2nd column
over65.insert(1,'Region', Region, allow_duplicates=False)

In [6]:
#Data transformations

# select numerical columns to convert to floats
claimsData[cols] = claimsData[cols].apply(pd.to_numeric, errors='coerce')

# removes any spaces in the string
claimsData['NPPES Provider State Description'] = claimsData['NPPES Provider State Description'].str.strip()

# applys HCPCSgroup() fcn to the 'HCPCS Code' array and creates a new 'category' array
claimsData['category'] = claimsData.apply(lambda x: HCPCSgroup(x['HCPCS Code']), axis=1) 

# applys region() fcn to 'NPPES Provider State Description' array and creates a new 'Region' array
claimsData['Region'] = claimsData.apply(lambda x: region(x['NPPES Provider State Description']), axis=1)

# multiplying 2 arrays together to create a new array that contains the total cost spent by Medicare on a service
claimsData['Total Medicare Payment'] = claimsData['Average Medicare Payment Amount'] * claimsData['Number of Services']

# multiplying 2 arrays together to create a new array that contains the total amount charged on a service
claimsData['Total Submitted Charge'] = claimsData['Average Submitted Charge Amount'] * claimsData['Number of Services'] 



In [7]:
# writes the file to a .csv
claimsData.to_csv('2012-2017_Medicare.csv', index=False)
under65.to_csv('2012-2017 Under_65.csv', index=False)
over65.to_csv('2012-2017 Over_65.csv', index=False)